In [ ]:
import cv2
import os
import numpy as np
import random
from tqdm import tqdm

class SyntheticDatasetGenerator:
    def __init__(self, target_images=5000):
        self.target_images = target_images
        self.dataset_dir = 'synthetic_dataset'
        self.expressions = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
        self.img_size = 48
        
        # Create directories
        self.setup_directories()
    
    def setup_directories(self):
        """Create dataset directory structure"""
        print("📁 Creating dataset directories...")
        os.makedirs(self.dataset_dir, exist_ok=True)
        for expression in self.expressions:
            expr_dir = os.path.join(self.dataset_dir, expression)
            os.makedirs(expr_dir, exist_ok=True)
        print(f"✅ Created directories for {len(self.expressions)} expressions")
    
    def create_face_like_image(self, expression):
        """Create synthetic face-like images with expression characteristics"""
        # Create base face (ellipse)
        image = np.zeros((self.img_size, self.img_size, 3), dtype=np.uint8)
        center_x, center_y = self.img_size // 2, self.img_size // 2
        
        # Face color (skin tones)
        skin_tones = [
            (255, 220, 177),  # Light
            (255, 194, 138),  # Medium light
            (255, 165, 110),  # Medium
            (210, 140, 90),   # Medium dark
            (165, 110, 70)    # Dark
        ]
        face_color = random.choice(skin_tones)
        
        # Draw face
        face_radius = random.randint(18, 22)
        cv2.ellipse(image, (center_x, center_y), (face_radius, face_radius), 0, 0, 360, face_color, -1)
        
        # Add expression-specific features
        if expression == 'angry':
            image = self.add_angry_features(image, center_x, center_y)
        elif expression == 'disgust':
            image = self.add_disgust_features(image, center_x, center_y)
        elif expression == 'fear':
            image = self.add_fear_features(image, center_x, center_y)
        elif expression == 'happy':
            image = self.add_happy_features(image, center_x, center_y)
        elif expression == 'sad':
            image = self.add_sad_features(image, center_x, center_y)
        elif expression == 'surprise':
            image = self.add_surprise_features(image, center_x, center_y)
        else:  # neutral
            image = self.add_neutral_features(image, center_x, center_y)
        
        # Add noise and variations
        image = self.add_variations(image)
        
        return image
    
    def add_angry_features(self, image, center_x, center_y):
        """Add angry expression features"""
        # Angry eyebrows (downward slanted)
        brow_y = center_y - 8
        cv2.line(image, (center_x-12, brow_y-2), (center_x-4, brow_y), (0,0,0), 2)
        cv2.line(image, (center_x+4, brow_y), (center_x+12, brow_y-2), (0,0,0), 2)
        
        # Angry mouth (straight or slightly downward)
        mouth_y = center_y + 8
        cv2.line(image, (center_x-8, mouth_y), (center_x+8, mouth_y), (0,0,0), 2)
        
        return image
    
    def add_disgust_features(self, image, center_x, center_y):
        """Add disgust expression features"""
        # Wrinkled nose
        cv2.line(image, (center_x-2, center_y-2), (center_x+2, center_y-2), (0,0,0), 1)
        
        # Asymmetrical mouth
        mouth_points = np.array([
            [center_x-6, center_y+6],
            [center_x-2, center_y+8],
            [center_x+6, center_y+6]
        ], np.int32)
        cv2.polylines(image, [mouth_points], False, (0,0,0), 2)
        
        return image
    
    def add_fear_features(self, image, center_x, center_y):
        """Add fear expression features"""
        # Wide eyes
        cv2.circle(image, (center_x-8, center_y-4), 3, (0,0,0), -1)
        cv2.circle(image, (center_x+8, center_y-4), 3, (0,0,0), -1)
        
        # Open mouth (oval)
        cv2.ellipse(image, (center_x, center_y+8), (4, 3), 0, 0, 360, (0,0,0), 2)
        
        return image
    
    def add_happy_features(self, image, center_x, center_y):
        """Add happy/smile expression features"""
        # Smiling mouth (upward curve)
        cv2.ellipse(image, (center_x, center_y+8), (6, 3), 0, 0, 180, (0,0,0), 2)
        
        # Happy eyes (slightly curved)
        cv2.ellipse(image, (center_x-8, center_y-4), (2, 1), 0, 0, 180, (0,0,0), 1)
        cv2.ellipse(image, (center_x+8, center_y-4), (2, 1), 0, 0, 180, (0,0,0), 1)
        
        return image
    
    def add_sad_features(self, image, center_x, center_y):
        """Add sad expression features"""
        # Downward mouth
        cv2.ellipse(image, (center_x, center_y+10), (5, 2), 0, 180, 360, (0,0,0), 2)
        
        # Sad eyebrows
        brow_y = center_y - 8
        cv2.line(image, (center_x-12, brow_y+1), (center_x-4, brow_y), (0,0,0), 2)
        cv2.line(image, (center_x+4, brow_y), (center_x+12, brow_y+1), (0,0,0), 2)
        
        return image
    
    def add_surprise_features(self, image, center_x, center_y):
        """Add surprise expression features"""
        # Wide open eyes
        cv2.circle(image, (center_x-8, center_y-4), 4, (255,255,255), -1)
        cv2.circle(image, (center_x-8, center_y-4), 2, (0,0,0), -1)
        cv2.circle(image, (center_x+8, center_y-4), 4, (255,255,255), -1)
        cv2.circle(image, (center_x+8, center_y-4), 2, (0,0,0), -1)
        
        # Round open mouth
        cv2.circle(image, (center_x, center_y+8), 3, (0,0,0), 2)
        
        return image
    
    def add_neutral_features(self, image, center_x, center_y):
        """Add neutral expression features"""
        # Simple eyes
        cv2.circle(image, (center_x-8, center_y-4), 2, (0,0,0), -1)
        cv2.circle(image, (center_x+8, center_y-4), 2, (0,0,0), -1)
        
        # Straight mouth
        cv2.line(image, (center_x-6, center_y+8), (center_x+6, center_y+8), (0,0,0), 2)
        
        return image
    
    def add_variations(self, image):
        """Add random variations to make images unique"""
        # Random brightness variation
        brightness = random.uniform(0.8, 1.2)
        image = cv2.convertScaleAbs(image, alpha=brightness, beta=random.randint(-10, 10))
        
        # Random noise
        noise = np.random.randint(-5, 5, image.shape, dtype=np.int16)
        image = np.clip(image.astype(np.int16) + noise, 0, 255).astype(np.uint8)
        
        # Random blur
        if random.random() > 0.7:
            image = cv2.GaussianBlur(image, (3, 3), 0)
        
        return image
    
    def generate_dataset(self):
        """Generate the complete dataset"""
        print("🚀 GENERATING SYNTHETIC FACIAL EXPRESSION DATASET")
        print("=" * 60)
        print(f"🎯 Target: {self.target_images:,} images per expression")
        print(f"📊 Total: {self.target_images * len(self.expressions):,} images")
        print(f"💾 Location: {self.dataset_dir}/")
        
        total_generated = 0
        
        for expression in self.expressions:
            print(f"\n🎭 Generating {expression} images...")
            expr_dir = os.path.join(self.dataset_dir, expression)
            
            for i in tqdm(range(self.target_images), desc=f"  {expression}"):
                # Generate synthetic image
                image = self.create_face_like_image(expression)
                
                # Convert to grayscale (like real face images)
                gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                
                # Save image
                filename = f"{expression}_{i:05d}.jpg"
                filepath = os.path.join(expr_dir, filename)
                cv2.imwrite(filepath, gray_image)
                
                total_generated += 1
            
            print(f"✅ {expression}: {self.target_images} images generated")
        
        print("\n" + "=" * 60)
        print("🎉 DATASET GENERATION COMPLETE!")
        print(f"📊 Total images generated: {total_generated:,}")
        print(f"📁 Dataset location: {self.dataset_dir}/")
        
        # Verify the dataset
        self.verify_dataset()
    
    def verify_dataset(self):
        """Verify that all folders have the correct number of images"""
        print("\n🔍 Verifying dataset...")
        
        for expression in self.expressions:
            expr_dir = os.path.join(self.dataset_dir, expression)
            images = [f for f in os.listdir(expr_dir) if f.endswith('.jpg')]
            count = len(images)
            status = "✅" if count == self.target_images else "❌"
            print(f"   {status} {expression}: {count}/{self.target_images} images")
        
        print("✅ Dataset verification complete!")

def main():
    print("🎭 SYNTHETIC FACIAL EXPRESSION DATASET GENERATOR")
    print("This will create a dataset with 5000 images per expression automatically!")
    
    # Get target number
    try:
        target = int(input(f"Enter target images per expression (default 5000): ") or "5000")
    except:
        target = 5000
    
    # Generate dataset
    generator = SyntheticDatasetGenerator(target_images=target)
    generator.generate_dataset()

if __name__ == "__main__":
    main()

🎭 SYNTHETIC FACIAL EXPRESSION DATASET GENERATOR
This will create a dataset with 5000 images per expression automatically!
Enter target images per expression (default 5000): 5000
📁 Creating dataset directories...
✅ Created directories for 7 expressions
🚀 GENERATING SYNTHETIC FACIAL EXPRESSION DATASET
🎯 Target: 5,000 images per expression
📊 Total: 35,000 images
💾 Location: synthetic_dataset/

🎭 Generating angry images...


  angry: 100%|████████████████████████████████████████████████████████████████████| 5000/5000 [00:22<00:00, 226.79it/s]


✅ angry: 5000 images generated

🎭 Generating disgust images...


  disgust: 100%|██████████████████████████████████████████████████████████████████| 5000/5000 [00:29<00:00, 167.79it/s]


✅ disgust: 5000 images generated

🎭 Generating fear images...


  fear:  92%|███████████████████████████████████████████████████████████████▎     | 4585/5000 [00:30<00:01, 219.68it/s]